In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import csv
import matplotlib as mpl

In [ ]:
data=torch.load('complete_class_accuracy5shots',map_location=torch.device('cpu'))
classes,acc = data[:,0,:,:],data[:,1,:,:]

Let us look at how much of an increase in accuracy we could expect if we always took the best projection

In [ ]:
baseline_runs=acc[0,:,:].mean(1)
best_per_run=acc[1:,:,:].mean(2).max(0)[0]
print((best_per_run-baseline_runs).mean().item()*100)
print(torch.argsort(best_per_run-baseline_runs,descending=True))

The answer is a 3.24% increase in accuracy as a mean !

In [ ]:
for i in range(5):
    plt.plot(acc[:,i,0],label='run'+str(i))
plt.ylabel('accuracy')
plt.xlabel('projection')
plt.legend()
plt.title('Accuracy for the second class of one 5 way few shot problem')

In [ ]:
run=1
baseline=acc[0,run,:].mean()
plt.imshow(torch.cat((acc[:,run,:].mean(1).unsqueeze(-1),acc[:,run,:]),dim=1),aspect='auto')
plt.colorbar()
plt.xlabel('classes of the run number '+str(run)+ ' '+ str(classes[0,run].int().tolist()))
plt.ylabel('projections')
plt.title('Projection 0 corresponds to no projection. \nThe first column is the mean of other columns ')

In [ ]:
runs=5
color = plt.cm.rainbow(np.linspace(0, 1, runs))
for run, c in zip(range(0,3), color):
    baseline=acc[0,run,:].mean()
    plt.axhline(baseline,c=c)
    plt.plot(acc[:,run,:].mean(1),c=c,label='run'+str(run))
plt.title('run  0 to '+str(runs) + '\n projection 0 means no projection\n straight lines are baselines')
plt.ylabel('accuracy')
plt.xlabel('projection')
plt.legend()

Let us now see if there is any logic begind which projection is the good one. Let us look at the classes themselves.

In [ ]:
def extract_best_proj(acc,run,number_of_best=1):
    if number_of_best==1:
        return acc[:,run,:].mean(1).max().item(),acc[:,run,:].mean(1).argmax().item()-1
    else:
        sort=acc[:,run,:].mean(1).sort(descending=True)



def get_run(classes,acc,run,number_of_best=1):

    class_run=classes[0,run].int().tolist()

    baseline=acc[0,run,:].mean()

    best,best_class=extract_best_proj(acc,run,number_of_best)

    print(best)

    boostbest=best-baseline.repeat(number_of_best)

    print('best_class',best_class)

    print('boostbest',boostbest)

    print(class_run)

    return best_class,boostbest,class_run

print(get_run(classes,acc,4,2))



Etude du run 4 : La "best class to remove" (19) du training set est celle qui represente des "paresseux". Elle permet de gagner un boost de 5%. Les classes 11 2 19 16 8 du test set sont les classes "Bustes armure" "Grand chien (style golden retriever)" "Dessert à la fraise framboise vanille creme" "Pannaux affichage" "Furet". D'après le 2D plot plus haut dans le run 4 il y a des erreurs entre le chien et le furet au run 4. La classe à enlever est celle du "paresseux" car elle rapproche ces deux classes dans l'espace des features. La ressemblance évidente entre ces 3 animaux saute aux yeux !

Procédure pour choisir la classe à enlever : -repérer les couples de classes où il y a confusion (trop proche dans l'espace des features(à définir)) -prendre le vecteur moyen "m" de tous les exemples de ce couple de classe -repérer la classe (training set) la plus proche de "m" dans le linear layer du classifier de training set. -retirer cette contribution !

Résumé : parmis les 64 classes du training set certaines peuvent se situer "entre" 2 classes du run de few shot. En entrainant le backbone une classe ressere d'espace des features autour de son veteur caractéristique et crée la confusion pour les classes qui se situent de par et d'autre de cette même classe.

        return sort[0][:number_of_best],sort[1].int()[:number_of_best]-1

In [ ]:
def get_run(classes,acc,run,number_of_best=1):
    class_run=classes[0,run].int().tolist()
    baseline=acc[0,run,:].mean()
    best,best_class=extract_best_proj(acc,run,number_of_best)
    print(best)
    boostbest=best-baseline.repeat(number_of_best)
    print('best_class',best_class)
    print('boostbest',boostbest)
    print(class_run)
    return best_class,boostbest,class_run

In [ ]:
get_run(classes,acc,4,2)

Etude du run 4 : La "best class to remove" (19) du training set est celle qui represente des "paresseux". Elle permet de gagner un boost de 5%. Les classes 11 2 19 16 8 du test set sont les classes "Bustes armure" "Grand chien (style golden retriever)" "Dessert à la fraise framboise vanille creme" "Pannaux affichage" "Furet". D'après le 2D plot plus haut dans le run 4 il y a des erreurs entre le chien et le furet au run 4. La classe à enlever est celle du "paresseux" car elle rapproche ces deux classes dans l'espace des features. La ressemblance évidente entre ces 3 animaux saute aux yeux  ! 

Procédure pour choisir la classe à enlever : 
-repérer les couples de classes où il y a confusion (trop proche dans l'espace des features(à définir))
-prendre le vecteur moyen "m" de tous les exemples de ce couple de classe
-repérer la classe (training set) la plus proche de "m" dans le linear layer du classifier de training set.
-retirer cette contribution ! 

Résumé : parmis les 64 classes du training set certaines peuvent se situer "entre" 2 classes du run de few shot.
En entrainant le backbone une classe ressere d'espace des features autour de son veteur caractéristique et crée la confusion pour les classes qui se situent de par et d'autre de cette même classe.

In [ ]:
acc[34,4]-acc[0,4]

Il semblerait qu'enlever la contribution de la classe 33 (poele casserole) cause la confusion entre le chien et le buste d'armure. (reconnaissance d'objet metallique). J'aurais plutôt attendu des soucis entre la l'armure et le dessert (plat).

In [ ]:
import csv
filenametrain = '/home/r21lafar/Documents/dataset/miniimagenetimages/train.csv'
filenametest = '/home/r21lafar/Documents/dataset/miniimagenetimages/test.csv'
directory = '/home/r21lafar/Documents/dataset/miniimagenetimages/images/'

In [ ]:
def opencsv(filename):
    file = open(filename)
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)
    print(header)
    rowstrain = []
    rows = []
    for row in csvreader:
        rows.append(row)
    return rows

In [ ]:
test = opencsv(filenametest)
train = opencsv(filenametrain)

In [ ]:
def openimage(test,train,source, cl,title):
    if source=='test':
        src=test
    else:
        src=train
    if type(cl)==int:
        plt.figure(figsize=(5,5))
        idx=int((cl+0.5)*600)+np.random.randint(-100,100)
        filename=src[idx][0]
        im = Image.open(directory +filename)
        plt.title(title)
        plt.imshow(np.array(im))
    else:
        fig = plt.figure(figsize=(8,8))
        spec = mpl.gridspec.GridSpec(ncols=3, nrows=2)
        ax1 = fig.add_subplot(spec[0,0])   #classe 0
        ax2 = fig.add_subplot(spec[0,1])  #classe 1
        ax3 = fig.add_subplot(spec[0,2])  # classe 2
        ax4 = fig.add_subplot(spec[1,0])
        ax5 = fig.add_subplot(spec[1,1])
        L=[ax1,ax2,ax3,ax4,ax5]
        for i,classe in enumerate(cl):
            idx=int((classe+0.5)*600)+np.random.randint(-100,100)
            filename=src[idx][0]
            im = Image.open(directory +filename)
            L[i].imshow(np.array(im))
            L[i].set_title(str(i+1)+';'+str(classe))
    

In [ ]:
def run_images_interpolation(classes,acc,run,test,train,choice=0):
    best_class,boostbest,class_run = get_run(classes,acc,run,10)
    best_class=best_class[choice].item()
    boostbest=boostbest[choice].item()
    openimage(test,train,'train', best_class, 'best class to remove')
    openimage(test,train,'test', class_run, 'best class to remove')
    plt.figure()
    plt.plot(range(1,6),acc[best_class+1,run]-acc[0,run],'*',label='boost')
    #plt.plot(range(1,6),acc[0,run]/10,'*',label='baseline /10')
    plt.xlabel('class of few_shot')
    plt.ylabel('perf')
    plt.legend()
    print('CHOICE',best_class,boostbest)

In [ ]:
run=4
choice=0  #which of the first best would you like to remove
run_images_interpolation(classes,acc,run,test,train,choice)